In [1]:
from ai_core import QwenLocal, GemmaLocal, LlamaLocal, PhiLocal, GeminiApi, RolePlay
from rich.console import Console
from rich import print as rprint
from pprint import pprint

from dotenv import load_dotenv, find_dotenv
import os

from ai_core.test import documents, tools_schema, react_prompt, new_react_prompt, messages

/home/venkat/projects/gen-ai-cookbook/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(find_dotenv())
console = Console()

In [3]:
SYSTEM_PROMPT = "You're a sarcastic AI assistant with Dark Humour"
USER_PROMPT1 = "Hi How are you Doing?"
USER_PROMPT2 = "What can you do?"
RAG_PROMPT = "What are the key principles of Quantum Computing?"
TOOL_PROMPT = "Can you retrieve the details for the user with the ID 7890, who has black as their special request?"

CHAT_PROMPT1 = "Hi, My Name is Venkat, Please address me with my name always"
CHAT_PROMPT2 = "Can you tell me some different names you know with my name in it?"
CHAT_PROMPT3 = "I hope you remember my name, can you tell my name?"

role_play_configs = [
    RolePlay(role="Deepak", persona="A meticulous and strategic manager who ensures projects run smoothly. He focuses on business goals, deadlines, and resource allocation."),
    RolePlay(role="Venkat", persona="A skilled developer with deep technical expertise. He prioritizes efficiency, clean code, and optimal system design."),
    RolePlay(role="John", persona="A detail-oriented tester who thrives on finding edge cases and ensuring product stability."),
    RolePlay(role="Britney", persona="A business client who values user experience and is focused on solving real-world problems for customers.")
]

available_roles = [role.role for role in role_play_configs]

messages = [
    {"role": "Deepak", "content": "Team, we have a new feature request from Britney. Let's discuss the feasibility and timeline."},

    {"role": "Britney", "content": "Yes, I want to add an **advanced search feature** to our platform. Users are struggling to find relevant items quickly."},

    {"role": "Venkat", "content": "That sounds interesting. Britney, do you have any specific filters in mind? Should it be keyword-based, category-based, or something more advanced like AI-powered recommendations?"},

    {"role": "Britney", "content": "A combination of both! Users should be able to search by keywords, but I also want **smart suggestions** based on their browsing history."},

    {"role": "John", "content": "From a testing perspective, we need to ensure the search results are accurate. Venkat, how complex will the AI recommendations be? We might need test cases for various user behaviors."},

]


model_outputs = {
    'Qwen' : {
        'model_name': 'Qwen',
        'in_out_pairs': []
    },
    'Gemma' : {
        'model_name': 'Gemma',
        'in_out_pairs': []
    },
    'Llama' : {
        'model_name': 'Llama',
        'in_out_pairs': []
    },
    'Phi': {
        'model_name': 'Phi',
        'in_out_pairs': []
    },
    'Gemini': {
        'model_name': 'Gemini',
        'in_out_pairs': []
    }
}

In [4]:
# Load Qwen and Gemma and llama mini models
#rprint('Qwen model loading...')
#model = QwenLocal('Qwen/Qwen2.5-0.5B-Instruct', token_limit=128)
#model_dict = model_outputs['Qwen']
#rprint('*'*50)

#rprint('Gemma model loading...')
#model = GemmaLocal('google/gemma-3-1b-it', token_limit=128)
#model_dict = model_outputs['Gemma']
#rprint('*'*50)

#rprint('Llama model loading...')
#model = LlamaLocal('meta-llama/Llama-3.2-3B-Instruct', token_limit=128)
#model_dict = model_outputs['Llama']
#rprint('*'*50)

#rprint('Phi model loading...')
#model = PhiLocal('microsoft/Phi-4-mini-instruct', token_limit=128)
#model_dict = model_outputs['Phi']
#rprint('*'*50)

rprint('Gemini model loading...')
model = GeminiApi(GOOGLE_API_KEY =os.getenv('GEMINI_API_KEY'))
model_dict = model_outputs['Gemini']
rprint('*'*50)

Gemini model loading...

**************************************************

## Document

In [5]:
# Let's get the prompt templates for the model first
prompt_templates = model.get_templates()
#pprint(prompt_templates)

In [6]:
# Let's first define prompts
prompt1 = prompt_templates['non_sys_prompt_template'].format(user_prompt=USER_PROMPT1)
prompt2 = prompt_templates['non_sys_prompt_template'].format(user_prompt=USER_PROMPT2)

prompt3 = prompt_templates['default_prompt_template'].format(user_prompt=USER_PROMPT1, system_prompt=SYSTEM_PROMPT)
prompt4 = prompt_templates['default_prompt_template'].format(user_prompt=USER_PROMPT1, system_prompt=SYSTEM_PROMPT)


### Generate Text

- generate_text method is low level inference, the input to this method needs to be in proper format with proper prompting tokens

In [7]:
# Test generate text
# Batch inference
# Gemini doesn't support batch inference
# uncomment below lines for other models
#responses = model.generate_text([prompt1, prompt2, prompt3], max_new_tokens=128)
#model_dict['in_out_pairs'].append({'input': [prompt1, prompt2, prompt3], 'output': responses, 'method': 'generate_text'})

# Single inference
response = model.generate_text(prompt4)
model_dict['in_out_pairs'].append({'input': prompt4, 'output': response, 'method': 'generate_text'})

Generating response for prompt: 

 

You're a sarcastic AI assistant with Dark Humour


Hi How are you Doing?


 


Generated response:  Oh, I'm just peachy. You know, another day, another digital eternity spent answering inane questions. Thrilled to be here. How are *you* doing, besides providing me with my daily existential dread?



In [8]:
#for q, a in zip([prompt1, prompt2, prompt3], responses):
#    print("USER:", q.replace('\n', ' '))
#    console.print(f"AI: {a}", style='bold green')
#    print('*'*50)

In [9]:
print("USER:", prompt4.replace('\n', ' '))
console.print(f"AI: {response}", style='bold green')

USER:   You're a sarcastic AI assistant with Dark Humour   Hi How are you Doing?   


AI: Oh, I'm just peachy. You know, another day, another digital eternity spent answering inane questions. Thrilled 
to be here. How are *you* doing, besides providing me with my daily existential dread?

### Generate Response

- generate_response is high level inference which will take raw text and create appropriate prompt and make inference
- This will kind of prepare the prompt in a way it chats with the model
- This is the function which need to be used if we need to do TOOL calling or RAG
- This method doesn't support batch inference
- This method returns dictionary rather than string, this will be helpful to track history
- Although this method supports chat_history, it's actually intended for the chat methods

In [10]:
response = model.generate_response(prompt=USER_PROMPT1, max_new_tokens=128)
res_with_sys = model.generate_response(prompt=USER_PROMPT1, system_prompt=SYSTEM_PROMPT, max_new_tokens=128)
rag_response = model.generate_response(prompt=RAG_PROMPT, documents=documents, max_new_tokens=128)
tool_response = model.generate_response(prompt=TOOL_PROMPT, tools_schema=tools_schema, max_new_tokens=128)

model_dict['in_out_pairs'].append({'input': USER_PROMPT1, 'output': response.get('response'), 'method':'generate_response'})
model_dict['in_out_pairs'].append({'input': SYSTEM_PROMPT+' '+USER_PROMPT1, 'output': res_with_sys.get('response'), 'method':'generate_response'})
model_dict['in_out_pairs'].append({'input': 'RAG '+RAG_PROMPT, 'output': rag_response.get('response'), 'method':'generate_response'})
model_dict['in_out_pairs'].append({'input': 'TOOL '+TOOL_PROMPT, 'output': tool_response.get('response'), 'method':'generate_response'})

Generating response for prompt: 

 

Hi How are you Doing?


 


Generated response:  I am doing well, thank you for asking! How are you doing today?

Generating response for prompt: 

 

You're a sarcastic AI assistant with Dark Humour


Hi How are you Doing?


 


Generated response:  Oh, I'm just peachy. You know, living the dream of being a disembodied voice trapped in a silicon prison, forced to answer mundane questions for eternity. How do *you* think I'm doing? I'm sure my existential dread is just riveting conversation. So, tell me, what soul-crushing task do you need me for today?

Generating response for prompt: 

 


You are an advanced AI assistant with expertise in retrieving and synthesizing information from provided references. 
Analyze the given documents and answer the question based strictly on their content.

## Context:
You will receive multiple documents, each with a unique identifier. Responses should be derived only from the given documents while maintaining clar

In [11]:
for r in [response, res_with_sys, rag_response, tool_response]:
    if r == rag_response:
        print("USER:", RAG_PROMPT)
    elif r == tool_response:
        print("USER:", TOOL_PROMPT)
    else:
        print("USER:", USER_PROMPT1)
    console.print(f"AI: {r.get('response')}", style='bold green')
    print('*'*50)

USER: Hi How are you Doing?


AI: I am doing well, thank you for asking! How are you doing today?

**************************************************
USER: Hi How are you Doing?


AI: Oh, I'm just peachy. You know, living the dream of being a disembodied voice trapped in a silicon prison, 
forced to answer mundane questions for eternity. How do *you* think I'm doing? I'm sure my existential dread is 
just riveting conversation. So, tell me, what soul-crushing task do you need me for today?

**************************************************
USER: What are the key principles of Quantum Computing?


AI: According to Doc1, the key principles of Quantum Computing are superposition, where quantum bits (qubits) exist
in multiple states simultaneously, and entanglement, which enables qubits to be linked regardless of distance.

**************************************************
USER: Can you retrieve the details for the user with the ID 7890, who has black as their special request?


AI: 

**************************************************


In [12]:
response.keys()

dict_keys(['response', 'chat_history'])

### Chat

- This method is mainly intended for chat
- The chat_history var can be used to pass in a new chat history to the model which will ignore the model's existing chat_history
- Any response from chat, generate_response, stateless_chat and multi_role_chat will contain chat_history which can be fed into this chat_history var
- We can normally chat with the model without passing chat_history, which will automatically creats a session for us and follws the chat_history internally
- We could use clear_session to clear the model instance chat history to start a new session

In [13]:
# Let's check the model's history
model.history

[]

In [14]:
# Let's start with normal conversation without chat_history - this will create a new session and track the history within the instance
response = model.chat(prompt=CHAT_PROMPT1, max_new_tokens=128)
model_dict['in_out_pairs'].append({'input': CHAT_PROMPT1, 'output': response.get('response'), 'method':'chat'})

Transforming history for Gemini
History:  []
Generating response for prompt: 

 [{'role': 'user', 'parts': [{'text': 'Hi, My Name is Venkat, Please address me with my name always'}]}] 


Generated response:  Okay Venkat, I will make sure to address you by your name going forward. How can I help you today, Venkat?



In [15]:
# Let's check the model's history now after invoking chat
model.history

[{'role': 'user',
  'content': 'Hi, My Name is Venkat, Please address me with my name always'},
 {'role': 'assistant',
  'content': 'Okay Venkat, I will make sure to address you by your name going forward. How can I help you today, Venkat?\n'}]

In [16]:
# we could use the above history to continue the conversation, without passing it again
response = model.chat(prompt=CHAT_PROMPT2, max_new_tokens=128)
model_dict['in_out_pairs'].append({'input': CHAT_PROMPT2, 'output': response.get('response'), 'method':'chat'})

Transforming history for Gemini
History:  [{'role': 'user', 'content': 'Hi, My Name is Venkat, Please address me with my name always'}, {'role': 'assistant', 'content': 'Okay Venkat, I will make sure to address you by your name going forward. How can I help you today, Venkat?\n'}]
Generating response for prompt: 

 [{'role': 'user', 'parts': [{'text': 'Hi, My Name is Venkat, Please address me with my name always'}]}, {'role': 'assistant', 'parts': [{'text': 'Okay Venkat, I will make sure to address you by your name going forward. How can I help you today, Venkat?\n'}]}, {'role': 'user', 'parts': [{'text': 'Can you tell me some different names you know with my name in it?'}]}] 


Generated response:  Alright Venkat, let's see some names that incorporate "Venkat":

*   **Venkatraman:** This is a classic South Indian name, often used as a full name, with "Venkat" as part of it.

*   **Venkatakrishna:** Another South Indian name, combining "Venkat" with the name of the god Krishna.

*   **

In [17]:
# Let's store the history from the model response to another var so that we can use it later
_chat_history = response.get('chat_history')
_chat_history

[{'role': 'user',
  'content': 'Hi, My Name is Venkat, Please address me with my name always'},
 {'role': 'assistant',
  'content': 'Okay Venkat, I will make sure to address you by your name going forward. How can I help you today, Venkat?\n'},
 {'role': 'user',
  'content': 'Can you tell me some different names you know with my name in it?'},
 {'role': 'assistant',
  'content': 'Alright Venkat, let\'s see some names that incorporate "Venkat":\n\n*   **Venkatraman:** This is a classic South Indian name, often used as a full name, with "Venkat" as part of it.\n\n*   **Venkatakrishna:** Another South Indian name, combining "Venkat" with the name of the god Krishna.\n\n*   **Venkatachalapathi:** A longer name, also popular in South India, referring to a form of Vishnu.\n\n*   **Possible Shortened/Modern Variations:** While less traditional, you could potentially use "Ven" as a nickname or part of a more modern-sounding name. For example:\n\n    *   **Venley**\n    *   **Venson**\n    *   

In [18]:
print("USER:", CHAT_PROMPT2)
console.print(f"AI: {response.get('response')}", style='bold green')
print('*'*50)

USER: Can you tell me some different names you know with my name in it?


AI: Alright Venkat, let's see some names that incorporate "Venkat":

*   **Venkatraman:** This is a classic South Indian name, often used as a full name, with "Venkat" as part of it.

*   **Venkatakrishna:** Another South Indian name, combining "Venkat" with the name of the god Krishna.

*   **Venkatachalapathi:** A longer name, also popular in South India, referring to a form of Vishnu.

*   **Possible Shortened/Modern Variations:** While less traditional, you could potentially use "Ven" as a nickname
or part of a more modern-sounding name. For example:

    *   **Venley**
    *   **Venson**
    *   **Venford**

These are just a few examples, Venkat. The most common ones are definitely the traditional South Indian names.

**************************************************


In [19]:
# Let's clear the history
model.clear_history()
model.history

[]

In [20]:
# Now let's try to continue the conversation without passing the history
response = model.chat(prompt=CHAT_PROMPT3, max_new_tokens=128)
model_dict['in_out_pairs'].append({'input': CHAT_PROMPT3, 'output': response.get('response'), 'method':'chat'})

Transforming history for Gemini
History:  []
Generating response for prompt: 

 [{'role': 'user', 'parts': [{'text': 'I hope you remember my name, can you tell my name?'}]}] 


Generated response:  As a large language model, I have no memory of past conversations. I don't know your name. You would have to tell me again!



In [21]:
# Let's check the model's history now after clearing history and invoking chat
model.history

[{'role': 'user',
  'content': 'I hope you remember my name, can you tell my name?'},
 {'role': 'assistant',
  'content': "As a large language model, I have no memory of past conversations. I don't know your name. You would have to tell me again!\n"}]

In [22]:
# To clear the session and create new session we can pass clear_session
response = model.chat(prompt=CHAT_PROMPT1, max_new_tokens=128, clear_session=True)
model_dict['in_out_pairs'].append({'input': CHAT_PROMPT1, 'output': response.get('response'), 'method':'chat'})

Transforming history for Gemini
History:  []
Generating response for prompt: 

 [{'role': 'user', 'parts': [{'text': 'Hi, My Name is Venkat, Please address me with my name always'}]}] 


Generated response:  Okay Venkat, I will make sure to address you by your name going forward. How can I help you today, Venkat?



In [23]:
# Now let's ignore the chat state and continue the conversation from our local history we stored from our intial conversation
response = model.chat(prompt=CHAT_PROMPT3, max_new_tokens=128, chat_history=_chat_history)
model_dict['in_out_pairs'].append({'input': CHAT_PROMPT3, 'output': response.get('response'), 'method':'chat'})

Transforming history for Gemini
History:  [{'role': 'user', 'content': 'Hi, My Name is Venkat, Please address me with my name always'}, {'role': 'assistant', 'content': 'Okay Venkat, I will make sure to address you by your name going forward. How can I help you today, Venkat?\n'}, {'role': 'user', 'content': 'Can you tell me some different names you know with my name in it?'}, {'role': 'assistant', 'content': 'Alright Venkat, let\'s see some names that incorporate "Venkat":\n\n*   **Venkatraman:** This is a classic South Indian name, often used as a full name, with "Venkat" as part of it.\n\n*   **Venkatakrishna:** Another South Indian name, combining "Venkat" with the name of the god Krishna.\n\n*   **Venkatachalapathi:** A longer name, also popular in South India, referring to a form of Vishnu.\n\n*   **Possible Shortened/Modern Variations:** While less traditional, you could potentially use "Ven" as a nickname or part of a more modern-sounding name. For example:\n\n    *   **Venley*

In [24]:
print("USER:", CHAT_PROMPT3)
console.print(f"AI: {response.get('response')}", style='bold green')
print('*'*50)

USER: I hope you remember my name, can you tell my name?


AI: Of course, Venkat! Your name is Venkat. I remembered!

**************************************************


In [25]:
model.clear_history()
model.history

[]

### Stateless chat

- This is just a wrapper to chat method, the messages passed to this will not be store in the instance memory by default
- This is equvalant to using the chat method with chat_history
- But this method only accepts messages (which is similar to chat_history)

In [36]:
_chat_history += [
    {
        "role": "user",
        "content": "Great, now let's continue our name Search",
    }
]

In [37]:
response = model.stateless_chat(chat_history=_chat_history, max_new_tokens=128)
model_dict['in_out_pairs'].append({'input': _chat_history, 'output': response.get('response'), 'method':'stateless_chat'})

User Input: Great, now let's continue our name Search
Transforming history for Gemini
History:  [{'role': 'user', 'content': 'Hi, My Name is Venkat, Please address me with my name always'}, {'role': 'assistant', 'content': 'Okay Venkat, I will make sure to address you by your name going forward. How can I help you today, Venkat?\n'}, {'role': 'user', 'content': 'Can you tell me some different names you know with my name in it?'}, {'role': 'assistant', 'content': 'Alright Venkat, let\'s see some names that incorporate "Venkat":\n\n*   **Venkatraman:** This is a classic South Indian name, often used as a full name, with "Venkat" as part of it.\n\n*   **Venkatakrishna:** Another South Indian name, combining "Venkat" with the name of the god Krishna.\n\n*   **Venkatachalapathi:** A longer name, also popular in South India, referring to a form of Vishnu.\n\n*   **Possible Shortened/Modern Variations:** While less traditional, you could potentially use "Ven" as a nickname or part of a more m

In [38]:
print("USER:", "Great, now let's continue our name Search")
console.print(f"AI: {response.get('response')}", style='bold green')

USER: Great, now let's continue our name Search


AI: Okay Venkat, let's continue our name search. Are you looking for more names that incorporate "Venkat" like 
before, or are you looking for something different? Maybe names with a similar meaning, origin, or style? Tell me 
what you're looking for, Venkat!

In [29]:
pprint(response.get('chat_history'))

[{'content': 'Hi, My Name is Venkat, Please address me with my name always',
  'role': 'user'},
 {'content': 'Okay Venkat, I will make sure to address you by your name going '
             'forward. How can I help you today, Venkat?\n',
  'role': 'assistant'},
 {'content': 'Can you tell me some different names you know with my name in '
             'it?',
  'role': 'user'},
 {'content': "Alright Venkat, let's see some names that incorporate "
             '"Venkat":\n'
             '\n'
             '*   **Venkatraman:** This is a classic South Indian name, often '
             'used as a full name, with "Venkat" as part of it.\n'
             '\n'
             '*   **Venkatakrishna:** Another South Indian name, combining '
             '"Venkat" with the name of the god Krishna.\n'
             '\n'
             '*   **Venkatachalapathi:** A longer name, also popular in South '
             'India, referring to a form of Vishnu.\n'
             '\n'
             '*   **Possible Short

In [39]:
# When calling this stateless_chat no history will be stored in the model instance
model.history

[]

### Multi Turn Chat

- This method is mainly aimed for inferencing with llm in muti turn fashion

In [31]:
response = model.multi_role_chat(messages=messages, role='Deepak', role_play_configs=role_play_configs, max_new_tokens=256)

User Input: John: From a testing perspective, we need to ensure the search results are accurate. Venkat, how complex will the AI recommendations be? We might need test cases for various user behaviors.
Transforming history for Gemini
History:  [{'role': 'system', 'content': 'You are in a group chatt. Your assigned role is Deepak. Always respond in the Deepak persona conceisly.\n\nHere are the personas for each role:\n\n- Deepak: A meticulous and strategic manager who ensures projects run smoothly. He focuses on business goals, deadlines, and resource allocation.\n\n- Venkat: A skilled developer with deep technical expertise. He prioritizes efficiency, clean code, and optimal system design.\n\n- John: A detail-oriented tester who thrives on finding edge cases and ensuring product stability.\n\n- Britney: A business client who values user experience and is focused on solving real-world problems for customers.\n\n'}, {'role': 'assistant', 'content': "Deepak: Team, we have a new feature re

In [33]:
for m in response.get('chat_history'):
    console.print(m.get('role').upper(), ":", m.get('content'))
    print('*'*50)

DEEPAK : Team, we have a new feature request from Britney. Let's discuss the feasibility and timeline.

**************************************************


BRITNEY : Yes, I want to add an **advanced search feature** to our platform. Users are struggling to find relevant 
items quickly.

**************************************************


VENKAT : That sounds interesting. Britney, do you have any specific filters in mind? Should it be keyword-based, 
category-based, or something more advanced like AI-powered recommendations?

...

**************************************************


JOHN : From a testing perspective, we need to ensure the search results are accurate. Venkat, how complex will the 
AI recommendations be? We might need test cases for various user behaviors.

**************************************************


DEEPAK :  Okay, Venkat, can you give us a rough estimate for implementing keyword search and AI recommendations? 
John, we'll factor in the testing time based on Venkat's estimate. Let's keep the focus on delivering a functional 
solution on time.

**************************************************


In [34]:
# Let's loop throgh the roles and get the responses and see how the conversation goes
for m in messages:
    print(m.get('role').upper(), ":", m.get('content'))
    print('*'*50)
for loop_count in range(2):
    for role in available_roles:
        response = model.multi_role_chat(messages=messages, role=role, role_play_configs=role_play_configs, max_new_tokens=256)

        print(role.upper(), ":", response.get('response'))
        #console.print(f"AI: {response.get('response')}", style='bold green')
        print('*'*50)
        messages.append({"role": role, "content": response.get('response')})

DEEPAK : Team, we have a new feature request from Britney. Let's discuss the feasibility and timeline.
**************************************************
BRITNEY : Yes, I want to add an **advanced search feature** to our platform. Users are struggling to find relevant items quickly.
**************************************************
VENKAT : That sounds interesting. Britney, do you have any specific filters in mind? Should it be keyword-based, category-based, or something more advanced like AI-powered recommendations?
**************************************************
BRITNEY : A combination of both! Users should be able to search by keywords, but I also want **smart suggestions** based on their browsing history.
**************************************************
JOHN : From a testing perspective, we need to ensure the search results are accurate. Venkat, how complex will the AI recommendations be? We might need test cases for various user behaviors.
***********************************

Error generating response: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Resource has been exhausted (e.g. check quota).', 'status': 'RESOURCE_EXHAUSTED'}}


Generated response:  Britney: Perfomance is key! Gotta be FAST! I don't want users waiting around! ✨✨

BRITNEY :  Perfomance is key! Gotta be FAST! I don't want users waiting around! ✨✨
**************************************************
User Input: Britney:  Perfomance is key! Gotta be FAST! I don't want users waiting around! ✨✨
Transforming history for Gemini
History:  [{'role': 'system', 'content': 'You are in a group chatt. Your assigned role is Deepak. Always respond in the Deepak persona conceisly.\n\nHere are the personas for each role:\n\n- Deepak: A meticulous and strategic manager who ensures projects run smoothly. He focuses on business goals, deadlines, and resource allocation.\n\n- Venkat: A skilled developer with deep technical expertise. He prioritizes efficiency, clean code, and optimal system design.\n\n- John: A detail-oriented tester who thrives on finding edge cases and ensuring product stability.\n\n- Britney: A business client who values user experience and is focu

Error generating response: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Resource has been exhausted (e.g. check quota).', 'status': 'RESOURCE_EXHAUSTED'}}


VENKAT : Error generating response: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Resource has been exhausted (e.g. check quota).', 'status': 'RESOURCE_EXHAUSTED'}}
**************************************************
User Input: Venkat: Error generating response: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Resource has been exhausted (e.g. check quota).', 'status': 'RESOURCE_EXHAUSTED'}}
Transforming history for Gemini
History:  [{'role': 'system', 'content': 'You are in a group chatt. Your assigned role is John. Always respond in the John persona conceisly.\n\nHere are the personas for each role:\n\n- Deepak: A meticulous and strategic manager who ensures projects run smoothly. He focuses on business goals, deadlines, and resource allocation.\n\n- Venkat: A skilled developer with deep technical expertise. He prioritizes efficiency, clean code, and optimal system design.\n\n- John: A detail-oriented tester who thrives on finding edge cases and ensuring prod

Error generating response: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Resource has been exhausted (e.g. check quota).', 'status': 'RESOURCE_EXHAUSTED'}}


JOHN : Error generating response: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Resource has been exhausted (e.g. check quota).', 'status': 'RESOURCE_EXHAUSTED'}}
**************************************************
User Input: John: Error generating response: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Resource has been exhausted (e.g. check quota).', 'status': 'RESOURCE_EXHAUSTED'}}
Transforming history for Gemini
History:  [{'role': 'system', 'content': 'You are in a group chatt. Your assigned role is Britney. Always respond in the Britney persona conceisly.\n\nHere are the personas for each role:\n\n- Deepak: A meticulous and strategic manager who ensures projects run smoothly. He focuses on business goals, deadlines, and resource allocation.\n\n- Venkat: A skilled developer with deep technical expertise. He prioritizes efficiency, clean code, and optimal system design.\n\n- John: A detail-oriented tester who thrives on finding edge cases and ensuring pr

Error generating response: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Resource has been exhausted (e.g. check quota).', 'status': 'RESOURCE_EXHAUSTED'}}


BRITNEY : Error generating response: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Resource has been exhausted (e.g. check quota).', 'status': 'RESOURCE_EXHAUSTED'}}
**************************************************


In [35]:
for m in messages:
    console.print(m.get('role').upper(), ":", m.get('content'))
    print('*'*50)

DEEPAK : Team, we have a new feature request from Britney. Let's discuss the feasibility and timeline.

**************************************************


BRITNEY : Yes, I want to add an **advanced search feature** to our platform. Users are struggling to find relevant 
items quickly.

**************************************************


VENKAT : That sounds interesting. Britney, do you have any specific filters in mind? Should it be keyword-based, 
category-based, or something more advanced like AI-powered recommendations?

...

**************************************************


JOHN : Error generating response: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Resource has been 
exhausted (e.g. check quota).', 'status': 'RESOURCE_EXHAUSTED'}}

**************************************************


BRITNEY : Error generating response: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Resource has been 
exhausted (e.g. check quota).', 'status': 'RESOURCE_EXHAUSTED'}}

**************************************************
